In [90]:
# Pkg.add("PlotlyJS")
using Statistics
using Plots
using Symbolics
using LinearAlgebra
using Printf
# using PGFPlotsX
using PlotlyJS
using Kal

# Helper Functions

In [53]:
# could also replace with decaying α

# function line_search(f,x,d)
#     objective = α -> f(x .+ α*d)
#     # print("obj ", objective)
#     bracket_minimum(objective, x)
#     α = minimize(objective, a, b)
#     return x .+ α*d 
# end

# # f([x1, x2, x3]) = sin(x1 * x2) + exp(x2 + x3) - x3
# f(x) = sin(x[1] * x[2]) + exp(x[2] + x[3]) - x[3]
function bracket_minimum(f, x=0, s=1e-2, k=2)
    """Used for univariate problems"""
    a, ya = x, f(x)
    b, yb = a .+ s, f(a .+ s)
    print("\n yb ", yb, " ya ", ya)
    if yb > ya
        # switch b to be moving closer to the minimum
        a, b = b, a
        ya, yb = yb, ya
        # switch direction we are moving in 
        s = -s
    end
    while true
        # moving b along, taking steps past b, and calling it c
        c, yc = b + s, f(b + s)
        if yc > yb
            # yc should be moving into the valley, so if its > b, probably going up a hill.. switch a and c
            return a < c ? (a, c) : (c, a)
        end
        # move all the vals over, a ->b, b -> c 
        a, ya, b, yb = b, yb, c, yc
        # double the step size
        s*=k
    end
end

function golden_section_search(f, a, b, n)
    "minimize univariate function, given an a bracket interval defined by a and b within n iterations"
    φ = MathConstants.golden
    ρ = φ - 1
    d = ρ * b + (1 - ρ)*a
    yd = f(d)
    for i = 1: n -1
        c = ρ*a + (1 - ρ)*b
        yc = f(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a,b = b, c
        end
    end
    return a < b ? (a,b) : (b,a)
end

function interval_middle(a,b)
    "get the value at the center of an interval"
    return (a+b)/2
end

function minimize(f, a, b, n=10)
    "golden section search returning center of an interval"
    low, high = golden_section_search(f, a, b, n)
    return interval_middle(low, high)
end

function line_search(f,x,d)
    objective = α -> f(x + α * d)
    # print(f(x), ' ', x, ' ', d, '\n')
    # ccall(:jl_exit, Cvoid, (Int32,), 86)
    a, b = bracket_minimum(objective)
    # print('\n', a, ' ', b)
    α = minimize(objective, a, b)
    # print(x + α * d, ' ',  x, ' ', α, ' ', d)
    return x + α * d 
end

function line_search_fix_alpha(f,x,d, iter=1)
    objective = α -> f(x + α * d)
    # print(f(x), ' ', x, ' ', d, '\n')
    # ccall(:jl_exit, Cvoid, (Int32,), 86)
    a, b = bracket_minimum(objective)
    # print('\n', a, ' ', b)
    α = minimize(objective, a, b)
    if iter > 1
        α = α
    end
    # print(x + α * d, ' ',  x, ' ', α, ' ', d)
    return x + α * d 
end



# minimize: to do implement brent dekker or golden section search 



# ccall(:jl_exit, Cvoid, (Int32,), 86)

# f(x) = sin(x[1] * x[2]) + exp(x[2] + x[3]) - x[3]

# line_search(f, [1,2,3], [0,-1,-1])
# line_search(f, [1,2,3], [0,-1,-1])

# d = 12
# α = 2
# objective = α -> f(x + α*d)

line_search (generic function with 1 method)

# BFGS Implementation

In [81]:
mutable struct BFGS
    Q #modifying matrix
end 

function init!(M::BFGS, x)
    # initialize matrix Q with the identity matrix
    # will have dimensions of m * m, wher m is the size of input vector x
    m = length(x)
    M.Q = Matrix(1.0*I, m, m)
    return M
end

init! (generic function with 1 method)

In [96]:
function step!(M::BFGS, f, ∇f, x)
        Q, g = M.Q, ∇f(x)
        # print("x: ", x, " Q: ", Q, ", g: ", ∇f(x))
        # print(" f: ", f, " x: ", x, " -Q*g ", -Q*g)
       
        x′ = line_search(f, x, -Q*g)
        # print("\n x′ 0 : ", x′, "\n")
        g′ = ∇f(x′)
        # print("\n g′ 0 : ", g′, "\n")

        δ = x′ - x
        γ = g′ - g
        # print("\n δ : ", δ, "\n")
        # print("\n γ : ", γ, "\n")
        # print("\n Q_init : ", Q, "\n")
        
        Q[:] = Q - (δ*γ'*Q + Q*γ*δ')/(δ'*γ) + (1 + (γ'*Q*γ)/(δ'*γ))[1] * (δ*δ')/(δ'*γ)

        # print("\n Q : ", Q, "\n")
        # print("x′ 1 : ", x′, "\n")
        return x′
end

step! (generic function with 1 method)

In [106]:
function quadratic(x)
    return x^2
end

function grad_quadratic(x)
    return 2*x 
end

f(x) = sin(x[1] * x[2]) + exp(x[2] + x[3]) - x[3]
f′(x) = [cos(x[1] + x[2]), cos(x[1] + x[2]) + exp(x[2] + x[3]), exp(x[2] + x[3]) - 1]


testM = BFGS(undef)
myx = [1,2,3]
myM = init!(testM, myx)
# step!(myM, quadratic, grad_quadratic, [2])
step!(myM, f, f′, myx)



 yb 6.761130123764031 ya 146.32245652940227

3-element Vector{Float64}:
  1.0210488700796805
 -1.1344591916403948
 -0.13424641564501893

# Optimization

In [108]:

"""
optimize(f, g, x0, n, prob)

Arguments:
- `f`: Function to be optimized
- `g`: Gradient function for `f`
- `x0`: (Vector) Initial position to start from
- `n`: (Int) Number of evaluations allowed. Remember `g` costs twice of `f`
- `prob`: (String) Name of the problem. So you can use a different strategy for each problem. E.g. "simple1", "secret2", etc.

Returns:
- The location of the minimum
"""
function optimize(f, g, x0, n, prob)
    # using a problem agnostic BFGS method
    # starting matrix to be initialized 
    undef_m = BFGS(undef)
    init_m = init!(undef_m, myx)

    # take a step with BFGS 
    x_best = step!(init_m, f, g, x0)
   
    return x_best
end

optimize

# Functions and their Gradients..

In [46]:
# just looking at the gradoent
@variables x y z
f′(x,y,z) = [cos(x + y), cos(x + y) + exp(y + z), exp(y + z) - 1]
# f′(x) = [cos(x[1] + x[2]), cos(x[1] + x[2]) + exp(x[2] + x[3]), exp(x[2] + x[3]) -1]
f′(x,y,z)

3-element Vector{Num}:
    cos(x + y)
 cos(x + y) + exp(y + z)
              exp(y + z) - 1

In [12]:
function rosenbrock(x::Vector)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

rosenbrock (generic function with 1 method)

# Plot

In [194]:
b = []
for i in range(1,10)
    push!(b, i)
    
end
b

10-element Vector{Any}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [146]:
# make contour 
my_range = range(-5, 5)

a = zeros(Float32, length(my_range), 2)
a[:,1] = 0.3*my_range
a[:,2] = 0.3*my_range
A = [rosenbrock([x1, x2]) for x1=a[:,1], x2=a[:,2]]

11×11 Matrix{Float64}:
 1412.5   1196.5   998.5   818.5   656.5   …  188.5       116.5    62.5
  869.2    701.8   552.4   421.0   307.6       34.0        10.6     5.2
  537.22   407.62  296.02  202.42  126.82       4.42       18.82   51.22
  348.52   245.92  161.32   94.72   46.12      31.72       73.12  132.52
  254.5    168.1    99.7    49.3    16.9       67.3       124.9   200.5
  226.0    145.0    82.0    37.0    10.0   …   82.0       145.0   226.0
  253.3    166.9    98.5    48.1    15.7       66.1       123.7   199.3
  346.12   243.52  158.92   92.32   43.72      29.32       70.72  130.12
  533.62   404.02  292.42  198.82  123.22       0.820001   15.22   47.62
  864.4    697.0   547.6   416.2   302.8       29.2         5.8     0.399999
 1406.5   1190.5   992.5   812.5   650.5   …  182.5       110.5    56.5

In [129]:
x_o = [-0.48993012686076465, 0.9627677544110731] 
x_best = [-0.7790026144206841, 0.11867994531271409]

x_el = [x_o[1], x_best[1]]
y_el = [x_o[2], x_best[2]]

2-element Vector{Float64}:
 0.9627677544110731
 0.11867994531271409

In [174]:
function make_trace(x_o, x_best)
    x_el = [x_o[1], x_best[1]]
    y_el = [x_o[2], x_best[2]]
    trace = PlotlyJS.scatter(x=x_el, y=y_el, mode="markers+lines", line_color="black", showlegend=false)
    return trace
end

make_trace (generic function with 1 method)

In [167]:
plotlyjs()

mycontour = PlotlyJS.contour(
    x=a[:,1],
    y=a[:,2],
    z=A,
    contours_coloring="lines",
    line_width=2,
    contours=attr(
        coloring ="heatmap",
        showlabels = true, # show labels on contours
        labelfont = attr( # label font properties
            size = 12,
            color = "white",
        )
    )
)

contour with fields contours, line, transpose, type, x, y, and z


In [180]:
trace0 = make_trace([-0.48993012686076465, 0.9627677544110731],[-0.7790026144206841, 0.11867994531271409] )

trace1 = make_trace([-0.2944566095565206, 1.0284222436351078], [-0.37239833237751757, 0.14658808251570787])

trace2 = make_trace([1.4175309969426104, -0.10287435201396039], [-0.07040497078645454, 0.007803260170605078])

layout = Layout(
    title="Rosenbrock’s function",
    xaxis_title="x1",
    yaxis_title="x2",
)


myplot = PlotlyJS.plot([mycontour, trace0, trace1, trace2
], layout)
# PlotlyJS.addtraces!(myplot, trace1)
myplot
# PlotlyJS.plot(t)


data: [
  "contour with fields contours, line, transpose, type, x, y, and z",
  "scatter with fields line, mode, showlegend, type, x, and y",
  "scatter with fields line, mode, showlegend, type, x, and y",
  "scatter with fields line, mode, showlegend, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/julietnwagwuume-ezeoke/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/julietnwagwuume-ezeoke/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66
┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/julietnwagwuume-ezeoke/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/julietnwagwuume-ezeoke/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66


In [109]:
# string interpolation 
x = 2
y = 4
"Hello $x, $(y^2) !"

"Hello 2, 16 !"

In [ ]:
data = rand(Float64, (100, 5))
r_range = 10:10:90


for idx in 1:size(data)[2]
    push!(traces, scatter(x=r_range,
                         y=data[:,idx],
                        mode="lines",
                        name = "$idx")
end

plot_size = 800

layout = Layout(
    xaxis_label = "x",
    yaxis_label = "y",
    font = attr(size=16),
    width=plot_size, height=plot_size,
    template = "plotly_white"
)


In [181]:
a = map(1:10) do aᵢ
    1 +1
end
a

10-element Vector{Int64}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2